<a href="https://colab.research.google.com/github/Pperezhogin/pyqg_generative/blob/master/Google-Colab/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Needed only on Google Colab
try:
    import google.colab
    !pip install aiohttp requests zarr pyfftw gcm_filters pyqg
    !pip install git+https://github.com/m2lines/pyqg_generative.git
    print('\n------------- Packages on Google Colab are installed ------------')
except:
    print('\n------------- Not running on Google Colab --------------')


------------- Not running on Google Colab --------------


In [2]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
def open_zarr(folder):
    for url, label in zip(['https://g-402b19.00888.8540.data.globus.org', 'https://g-771f59.00888.8540.data.globus.org'], ['archive', 'scratch']):
      try:
        mapper = fsspec.get_mapper(f'{url}/{folder}.zarr')
        return xr.open_zarr(mapper, consolidated=True)
      except:
        print(f'{folder} on {label} failed')

In [10]:
import json
def read_model(folder):
  with open(f'{folder}/model_args.json') as file:
    model_args = json.load(file)
    return eval(model_args.pop('model'))(folder=folder, **model_args)

In [11]:
ds = open_zarr('eddy/48/sharp')

In [12]:
test = ds.isel(run=slice(250,252), time=slice(-2,None)).load()

# Open pretrained models

In [13]:
from pyqg_generative.models.cgan_regression import CGANRegression
from pyqg_generative.models.cvae_regression import CVAERegression
from pyqg_generative.models.mean_var_model import MeanVarModel

In [15]:
gan = read_model('GAN')
vae = read_model('VAE')
gz = read_model('GZ')

reading CGANRegression G, D from GAN
reading CVAERegression from VAE
reading MeanVarModel mean from GZ
reading MeanVarModel var from GZ


In [18]:
gan_offline = gan.test_offline(test, ensemble_size=10)

/home/pp2681/pyqg_generative/pyqg_generative/tools/spectral_tools.py:95: RuntimeWarning: Mean of empty slice.
  af2 = af2.mean(axis=(0,1))
/ext3/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/pp2681/pyqg_generative/pyqg_generative/tools/spectral_tools.py:95: RuntimeWarning: Mean of empty slice.
  af2 = af2.mean(axis=(0,1))
/ext3/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/pp2681/pyqg_generative/pyqg_generative/tools/spectral_tools.py:95: RuntimeWarning: Mean of empty slice.
  af2 = af2.mean(axis=(0,1))
/ext3/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/pp2681/pyqg_generative/pyqg_generative/tools/spectral_tools.py:95: RuntimeWarning: Mean of empty slice.
  af2 = af2

ValueError: supplied range of [nan, nan] is not finite